In [1]:
# %matplotlib inline
from astropy.time import Time
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from lsst_efd_client import EfdClient
from collections import defaultdict
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from scipy.stats import norm
import os

# M1M3 hardpoint repeatability and resolution analysis
---

## Overview
This notebook is dedicated to the analysis of M1M3 hardpoint repeatability.

Data to be analyzed are for the Requirement: "LVV-11200 LTS-88-REQ-0015-V-01: 3.7.1.3 Hardpoint Displacement Repeatability and Resolution_1" - https://jira.lsstcorp.org/browse/LVV-11200

The ticket related to this analysis - https://rubinobs.atlassian.net/browse/SITCOM-756

There are three key deliverables
- Plot Force vs. displacement for all actuators.
- Make a histogram of the displacement values per actuator.
- From the generated plot, measure the Hardpoint displacement repeatability (Average) and its resolution (FWHM) around 0 Newtons.

## Querying EFD

### Helper Function

In [2]:
async def query_bump_logs_in_chunks(
    start_date, end_date, client_name="", chunk_size_days=3,topic_name="lsst.sal.MTM1M3.logevent_logMessage",fields=["message"]
):
    """
    Queries the log messages related to bump tests from the EFD in chunks.

    Args:
        start_date (str): Start date of the query in ISO format (YYYY-MM-DD).
        
        end_date (str): End date of the query in ISO format (YYYY-MM-DD).
        
        client_name (str, optional): Name of the EFD client. Defaults to "".
        
        chunk_size_days (int, optional): Number of days per chunk. Defaults to 3.

        topic_name (str, optional): SAL topic name to be queried by the client. Defaults to lsst.sal.MTM1M3.logevent_logMessage.

        fields (list[str], optional): Fields to be queried by the client. Defaults to ["message"].

    Returns:
        pandas.DataFrame: Concatenated DataFrame containing the queried log messages.
    """

    client = makeClient(client_name)

    # Convert start and end dates to datetime objects
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)

    # Initialize an empty DataFrame to store concatenated results
    all_data = pd.DataFrame()

    current_start = start
    while current_start < end:
        current_end = min(current_start + timedelta(days=chunk_size_days), end)
        try:
            # Query the data for the current chunk
            chunk_data = await client.select_time_series(
                topic_name=topic_name,
                fields=fields,
                start=Time(current_start.isoformat(), format="isot", scale="utc"),
                end=Time(current_end.isoformat(), format="isot", scale="utc"),
            )
            # Concatenate the chunk data to the main DataFrame
            all_data = pd.concat([all_data, chunk_data], ignore_index=False)
        except Exception as e:
            print(
                f"Error querying data from {current_start.isoformat()} to {current_end.isoformat()}: {e}"
            )
            continue  # Optionally, continue to the next chunk

        # Move to the next chunk
        current_start = current_end

    return all_data

def makeClient(client_name):
        # Create the client based on client_name
    if client_name == "summit_efd":
        return makeEfdClient("summit_efd")
    elif client_name == "usdf_efd":
        return makeEfdClient("usdf_efd")
    elif client_name == "idf_efd":
        return makeEfdClient("idf_efd")
    else:
        return makeEfdClient()  # Default client


# Example usage:
# begin = "2023-11-13T01:00"
# end = "2023-12-21T01:00"
# bump_logs = await query_bump_logs_in_chunks(begin, end, client_name='')

def showAndClear():
    plt.show()
    # Clear the current axes.
    plt.cla() 
    # Clear the current figure.
    plt.clf() 
    # Closes all the figure windows.
    plt.close('all')   
    plt.close(fig)
    
    return

### Setting up the sub-directories

In [3]:
base_dir = os.getcwd()
figure_dir,data_dir = base_dir+"/SITCOM-756-plots",base_dir+"/SITCOM-756-data"
for pathname in [figure_dir,data_dir]:
    if not os.path.isdir(pathname):
        os.mkdir(pathname)

## Let's take a look at the topics, so that we can get an idea of what we need to query

In [4]:
client = makeClient("usdf_efd")

a = await client.get_topics() 

### Possibilities for the SAL data

- lsst.sal.MTM1M3.forceActuatorData
- lsst.sal.MTM1M3.forceActuatorPressure
- lsst.sal.MTM1M3.hardpointActuatorData -> This is where the data for the hardpoint is!
- lsst.sal.MTM1M3.hardpointMonitorData

In [5]:
starts = np.array(["2023-11-27T21:25:26","2023-11-28T20:40:38","2023-12-04T21:32:18","2023-12-07T03:34:44"]) # From https://lsstc.slack.com/archives/C0567AY64AC/p1715789391584609?thread_ts=1715786982.696169&cid=C0567AY64AC
ends = np.array(["2023-11-27T22:41:17","2023-11-28T21:56:28","2023-12-04T22:45:06","2023-12-07T04:48:29"])

In [6]:
colnames =  ["displacement","measuredForce"]
colors = ["#e01616","#f6f40b","#2affbb","#123cdf","#e51983","#2b8221"]

In [7]:
all_data = pd.DataFrame()
for start,end in zip(starts,ends):
    print(r"Starting query for time range {} - {}".format(start,end),end=" . . . ")
    df_bump = await client.select_time_series(
        "lsst.sal.MTM1M3.hardpointActuatorData", 
        "*", Time(start), Time(end))
    
    all_data = pd.concat([all_data, df_bump], ignore_index=False)
    
    print("Finished")
    del df_bump

Starting query for time range 2023-11-27T21:25:26 - 2023-11-27T22:41:17 . . . Finished
Starting query for time range 2023-11-28T20:40:38 - 2023-11-28T21:56:28 . . . Finished
Starting query for time range 2023-12-04T21:32:18 - 2023-12-04T22:45:06 . . . Finished
Starting query for time range 2023-12-07T03:34:44 - 2023-12-07T04:48:29 . . . Finished


### Plot Force vs. displacement for all actuators.
- Take displacement values from up and down movements for each actuator that is closest to Force = Zero Newton for all repetitions.
- Calculate the average.

In [8]:
for start,end in zip(starts,ends):
    fig,axs = plt.subplots(2,1,sharex=True,dpi=140)
    fig.suptitle(start[:10])
    mydf = all_data[np.logical_and(Time(all_data.index.values) >= Time(start),Time(all_data.index.values)<=Time(end))]
    for number in range(6):
        axs[0].plot(mydf.index.values,mydf[colnames[0]+str(number)],color=colors[number],
                    label=r"Actuator {}".format(str(number)),marker="x",ls='--')
        axs[1].plot(mydf.index.values,mydf[colnames[1]+str(number)],color=colors[number],
                    label=r"Actuator {}".format(str(number)),marker="+",ls='-.')
        
    axs[0].legend(ncols=2,fontsize='xx-small')
    axs[1].legend(ncols=2,fontsize='xx-small')
    axs[0].set_ylabel("Displacement [m]")
    axs[1].set_ylabel("Force [N]")
    for a in axs:
        a.xaxis.set_major_locator(mdates.MinuteLocator(interval = 15))
        a.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        a.grid()

    axs[1].set_xlabel("Date")

    fig.tight_layout()
    
    fig.savefig(r"{}/{}_actuator_m1m3.jpg".format(figure_dir,start))

    # showAndClear()

#### That looks okay, but it would be nice to mask these figures

In [9]:
for start,end in zip(starts,ends):
    fig,axs = plt.subplots(2,1,sharex=True,dpi=140)
    fig.suptitle(start[:10])
    mydf = all_data[np.logical_and(Time(all_data.index.values) >= Time(start),Time(all_data.index.values)<=Time(end))]
    for number in range(6):
        date_range = mydf[np.abs(np.diff(mydf[colnames[0]+str(number)],prepend=mydf[colnames[0]+str(number)].iloc[0])) != 0].index.values
        # first dated test for actuator 1 has one point that does not meet the above criteria for masking...
        # popping the last element off with this conditional
        if number == 1 and start==starts[0]:
            date_range = date_range[:-1]
            
        min_date, max_date = np.min(date_range),np.max(date_range)
        
        finaldf = mydf[np.logical_and(Time(mydf.index.values) >= Time(min_date),Time(mydf.index.values)<=Time(max_date))]

        
        axs[0].plot(finaldf.index.values,finaldf[colnames[0]+str(number)],color=colors[number],
                    label=r"Actuator {}".format(str(number)),marker="x",ls='--')
        axs[1].plot(finaldf.index.values,finaldf[colnames[1]+str(number)],color=colors[number],
                    label=r"Actuator {}".format(str(number)),marker="+",ls='-.')
        
    axs[0].set_ylabel("Displacement [m]")
    axs[1].set_ylabel("Force [N]")
    for a in axs:
        a.xaxis.set_major_locator(mdates.MinuteLocator(interval = 15))
        a.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        a.grid()
        a.legend(ncols=2,fontsize='xx-small')

    axs[1].set_xlabel("Time")

    fig.tight_layout()
    
    fig.savefig(r"{}/{}_actuator_m1m3_masked.jpg".format(figure_dir,start))

    # showAndClear()

### Make a histogram of the displacement values per actuator.

- Fit a Gaussian distribution and determine the FWHM
- Plot a limit lines at 2um

In [10]:
def fitGaussian(data,ax):
    mu, std = norm.fit(data) 
    
    xmin, xmax = ax.get_xlim()
    x = np.linspace(np.floor(xmin), np.ceil(xmax), int(10E4))
    p = norm.pdf(x, mu, std)

    return mu,std,p,x,xmin,xmax

def getFWHM_from_gaussian(sigma):
    return 2*np.sqrt(np.log(2)*2)*sigma

In [11]:
save_fig=True
save_data=True
bin_num = int(5E1)
xranges = [(-0.005,0.02),(-4000,4000)]
print_outputs = False # Flag to print the outputs of the figures
kwargs = dict(bins=bin_num, stacked=True, density=True,histtype='stepfilled')
comment_text = "\n The first row gives the different dates of the tests. Each successive entry corresponds to a different date.\n The second row gives the average of the data when fitted to a Gaussian.\n The third row is the rms of the gaussian fit.\n The fourth row is the FWHM of the gaussian fit."

In [12]:
for colname,xrange,axis_unit in zip(colnames,xranges, ["[m]","[N]"]):
    # first plot
    for start,end in zip(starts,ends):
        fig,ax = plt.subplots(dpi=140)
        mydf = all_data[np.logical_and(Time(all_data.index.values) >= Time(start),Time(all_data.index.values)<=Time(end))]
        fig.suptitle(start.__str__()[:10])
        ymax_final = 0
        mu_arr,std_arr = np.array([]),np.array([])
        
        for number in range(6):
            date_range = mydf[np.abs(np.diff(mydf[colname+str(number)],prepend=mydf[colname+str(number)].iloc[0])) != 0].index.values
            # first dated test for actuator 1 has one point that does not meet the above criteria for masking...
            # popping the last element off with this conditional
            if number == 1 and start==starts[0]:
                date_range = date_range[:-1]
                
            min_date, max_date = np.min(date_range),np.max(date_range)
            
            finaldf = mydf[np.logical_and(Time(mydf.index.values) >= Time(min_date),Time(mydf.index.values)<=Time(max_date))]
    
            # plot a histogram
    
            ax.hist(finaldf[colname+str(number)],bins=bin_num, density=True, alpha=0.8, color=colors[number])
    
            
            # Fit a gaussian to it 
    
            mu,std,p,x,xmin,xmax = fitGaussian(finaldf[colname+str(number)],ax)
    
            mu_arr,std_arr = np.append(mu_arr,mu),np.append(std_arr,std)
    
            ax.plot(x, p, linewidth=2,label="Actuator {}: $\mu$={:.2E}, $\sigma$={:.2E}".format(number,mu, std),color=colors[number])
    
        __,ymax = ax.get_ylim()
        # Plot vertical lines here
        for number in range(6):
            ax.vlines(mu_arr[number],0,1.2*ymax,color=colors[number],ls ='--')
            # Plot two additional vertical lines below
            for sig in range(1,3): # This might just be 2 micron, so wait to see what comes of it...
                ax.vlines(mu_arr[number]+std_arr[number]*sig,0,1.2*ymax_final,color=colors[number],ls ='--')
                ax.vlines(mu_arr[number]-std_arr[number]*sig,0,1.2*ymax_final,color=colors[number],ls ='--')
            
        ax.grid()
        ax.set_ylabel("Counts / bin width ({:.2E})".format(np.diff(xrange)[0]/bin_num))
        ax.set_xlabel("Displacement [m]")
        # ax.set_title(r"{} - {}".format(start,end))
        ax.set_xlim(-6*std_arr.max(),6*std_arr.max())
        ax.legend(loc="upper right")
        ax.set_ylim(bottom=0)
        fig.tight_layout()

        if save_fig:
            fig.savefig(r"{}/{}_actuator_m1m3_histogram_allActuators_{}.jpg".format(figure_dir,start,colname))
    
        showAndClear()
    
    ### Repeat the above plot, but now have the figs be of the individual actuators and the data from their runs
    if print_outputs:
        print("For {}".format(colname))
    for actuator_num in range(6):
        fig,ax = plt.subplots(dpi=140)
        mu_arr,std_arr = np.array([]),np.array([])
        actuatorDF = pd.DataFrame()
        color_iter = 0
        if print_outputs:
            print("Actuator {}".format(actuator_num))
            print("Test Date","mu\t","sigma\t","FWHM",sep="\t | \t")
        
        for start,end in zip(starts,ends):
            mydf = all_data[np.logical_and(Time(all_data.index.values) >= Time(start),Time(all_data.index.values)<=Time(end))]
    
            date_range = mydf[np.abs(np.diff(mydf[colname+str(actuator_num)],prepend=mydf[colname+str(actuator_num)].iloc[0])) != 0].index.values
            # first dated test for actuator 1 has one point that does not meet the above criteria for masking...
            # popping the last element off with this conditional
            if actuator_num == 1 and start==starts[0]:
                date_range = date_range[:-1]
                
            min_date, max_date = np.min(date_range),np.max(date_range)
            finaldf = mydf[np.logical_and(Time(mydf.index.values) >= Time(min_date),Time(mydf.index.values)<=Time(max_date))]
    
            # plot a histogram
            ax.hist(finaldf[colname+str(actuator_num)], alpha=0.4, color=colors[color_iter], range=xrange,zorder=1, **kwargs)
            
            # Fit a gaussian to it 
            mu,std,p,x,xmin,xmax = fitGaussian(finaldf[colname+str(actuator_num)],ax)
            ax.plot(x, p,label="{}: $\mu$={:.2E}, $\sigma$={:.2E}".format(min_date.__str__()[:10],mu, std), linewidth=2,color=colors[color_iter])
            mu_arr,std_arr = np.append(mu_arr,mu),np.append(std_arr,std)
        
            actuatorDF = pd.concat([finaldf,actuatorDF],axis=0)
            color_iter+=1
            if print_outputs:
                print(min_date.__str__()[:10],"{:.2E}".format(mu),"{:.2E}".format(std),"{:.2E}".format(getFWHM_from_gaussian(std)),sep="\t | \t")
    
        # plot additional histogram for the combined dataset
        n,bins,patches = ax.hist(actuatorDF[colname+str(actuator_num)],color=colors[color_iter],alpha=0.4,  range=xrange,zorder=4, **kwargs)
        
        # Fit a gaussian to it 
        mu,std,p,x,xmin,xmax = fitGaussian(actuatorDF[colname+str(actuator_num)],ax)
        if print_outputs:
            print("Combined","{:.2E}".format(mu),"{:.2E}".format(std),"{:.2E}".format(getFWHM_from_gaussian(std)),sep="\t | \t",end="\n\n")
        
        ax.plot(x, p, linewidth=2,label="{}: $\mu$={:.2E}, $\sigma$={:.2E}".format("Combined",mu, std),color=colors[color_iter])
        mu_arr,std_arr = np.append(mu_arr,mu),np.append(std_arr,std)

        FWHM_arr = getFWHM_from_gaussian(std_arr)
        
        __,ymax_final = ax.get_ylim()
        
        # Plot vertical lines here
        vr = 0
        for mu,std in zip(mu_arr,std_arr):
            ax.vlines(mu,0,1.2*ymax_final,color=colors[number],ls ='--')
            # Plot two additional vertical lines below
            for sig in range(1,3): # This might just be 2 micron, so wait to see what comes of it...
                ax.vlines(mu+std*sig,0,1.2*ymax_final,color=colors[vr],ls ='--')
                ax.vlines(mu-std*sig,0,1.2*ymax_final,color=colors[vr],ls ='--')
            vr += 1
        del vr
        
        ax.grid()
        ax.set_ylabel("Counts / bin width ({:.2E})".format(np.diff(xrange)[0]/bin_num))
        ax.set_xlabel("{} {}".format(colname,axis_unit))
        ax.set_title(r"Actuator {}".format(actuator_num))
        ax.set_xlim(xrange)
        ax.legend(loc="upper left",fontsize='x-small')
        ax.set_ylim(0,ymax_final)
        fig.tight_layout()

        if save_fig:
            fig.savefig(r"{}/m1m3_histogram_actuator_{}_{}.jpg".format(figure_dir,actuator_num,colname))

        if save_data and not os.path.isfile(data_dir+"/"+colname+str(actuator_num)+".txt"):
            np.savetxt(data_dir+"/"+colname+str(actuator_num)+".txt", [np.append(list([x[:10] for x in starts]),"Combined"),FWHM_arr,std_arr,mu_arr],fmt="%s",footer = comment_text)
    
        showAndClear()

# From the generated plot, measure the Hardpoint displacement repeatability (Average) and its resolution (FWHM).

- See outputs from fitting when setting `print_outputs=True`
- Figures are located in `notebooks_vandv/notebooks/tel_and_site/subsys_req_ver/m1m3/SITCOM-756-plots`
- Data is located in `notebooks_vandv/notebooks/tel_and_site/subsys_req_ver/m1m3/SITCOM-756-data` using the schema `{displacement/measuredForce}{actuator number}.txt` for the data files

- The actuators are very consistent across the four dated tests. The repeatability for displacement by 10µm, 60µm, 10µm, 480µm, 10µm, and 170µm for actuators 0-5 over the four dated tests. All six actuators have a displacement FWHM ~1 cm.
- Fitting a gaussian to the force is ineffective, since the force measurements are bimodal. If the force measurements for each actuator were masked further, the fitting might be more insightful


| **Combined Data from Four Dated Tests** | **$\mu_{Displacement}$ [m]** | **$FWHM_{Displacement}$ [m]** | **$\mu_{Force}$ [N]** | **$FWHM_{Force}$ [N]** |
|-----------------------------------------|----------------------------|-----------------------------|---------------------|----------------------|
| **Actuator 0**                          | 6.11E-3                    | 9.75E-3                     | -2.57E3             | 5.09E3               |
| **Actuator 1**                          | 6.87E-3                    | 1.07E-2                     | -2.89E3             | 4.26E3               |
| **Actuator 2**                          | 6.29E-3                    | 9.42E-3                     | 1.15E3              | 6.66E3               |
| **Actuator 3**                          | 6.14E-3                    | 9.32E-3                     | 1.56E3              | 6.26E3               |
| **Actuator 4**                          | 7.01E-3                    | 1.11E-2                     | -1.66E3             | 6.52E3               |
| **Actuator 5**                          | 6.22E-3                    | 9.61E-3                     | 2.24E3              | 4.80E3               |